In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

df = pd.read_csv("../input/india-ml-hiring-av/train.csv")

# Any results you write to the current directory are saved as output.

In [2]:
train_y = df['m13']
df.drop(columns = 'm13',inplace = True)
df.columns

Index(['loan_id', 'source', 'financial_institution', 'interest_rate',
       'unpaid_principal_bal', 'loan_term', 'origination_date',
       'first_payment_date', 'loan_to_value', 'number_of_borrowers',
       'debt_to_income_ratio', 'borrower_credit_score', 'loan_purpose',
       'insurance_percent', 'co-borrower_credit_score', 'insurance_type', 'm1',
       'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12'],
      dtype='object')

In [3]:
df.dtypes

loan_id                       int64
source                       object
financial_institution        object
interest_rate               float64
unpaid_principal_bal          int64
loan_term                     int64
origination_date             object
first_payment_date           object
loan_to_value                 int64
number_of_borrowers         float64
debt_to_income_ratio        float64
borrower_credit_score       float64
loan_purpose                 object
insurance_percent           float64
co-borrower_credit_score    float64
insurance_type              float64
m1                            int64
m2                            int64
m3                            int64
m4                            int64
m5                            int64
m6                            int64
m7                            int64
m8                            int64
m9                            int64
m10                           int64
m11                           int64
m12                         

In [4]:
# months = ['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12']
# list_nonobj = [df.dtypes.keys()[item] for item in range(len(df.dtypes.keys())) if df.dtypes[item]!='object' and df.dtypes.keys()[item] not in months]
# list_nonobj
# df2 = df[list_nonobj]

In [5]:
# list_nonobj

In [6]:
# scaler = StandardScaler().fit(df2)
# df2 = scaler.transform(df2)
# df2 = pd.DataFrame(df2, columns = list_nonobj)

In [7]:
def bank_dependency(df,status):
    dict = {}

    for i in range(len(train_y)):
        if train_y[i]==status:
            if df['financial_institution'][i] not in dict.keys():
                dict[df['financial_institution'][i]] = 1
            else:
                dict[df['financial_institution'][i]] = dict[df['financial_institution'][i]] + 1

    return dict

In [8]:
dict1 = bank_dependency(df,1)
labels = list(dict1.keys())
values = list(dict1.values())

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

This means that almost half of the loans that fail are being done by OTHER banks. Now let us look at the vice versa stat

In [9]:
dict2 = bank_dependency(df,0)
labels = list(dict2.keys())
values = list(dict2.values())

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

This means that almost half of the loans that fail are also being done by OTHER banks.

In [10]:
def response_time(df):
    diff = []
    for i in range(len(df['origination_date'])):
        temp = int(df['first_payment_date'][i].split('/')[0]) - int(df['origination_date'][i].split('-')[1])
        diff.append(temp)
    df.drop(columns = ['origination_date','first_payment_date'],inplace = True)
    df['response_time'] = diff
    return df

We attempt to store the difference between loan origin and first loan payment

In [11]:
def one_hot(df,columns,prefix):
#     columns_encoding = ['source','financial_institution','loan_purpose']
    columns_encoding = columns
    df2 = pd.get_dummies(df,columns = columns_encoding, prefix = ['source','fi','lp'])
    return df2

In [12]:
df2 = response_time(df)
df2 = one_hot(df,['source','financial_institution','loan_purpose'],['source','fi','lp'])
df2.columns

Index(['loan_id', 'interest_rate', 'unpaid_principal_bal', 'loan_term',
       'loan_to_value', 'number_of_borrowers', 'debt_to_income_ratio',
       'borrower_credit_score', 'insurance_percent',
       'co-borrower_credit_score', 'insurance_type', 'm1', 'm2', 'm3', 'm4',
       'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12', 'response_time',
       'source_X', 'source_Y', 'source_Z', 'fi_Anderson-Taylor',
       'fi_Browning-Hart', 'fi_Chapman-Mcmahon', 'fi_Cole, Brooks and Vincent',
       'fi_Edwards-Hoffman', 'fi_Martinez, Duffy and Bird',
       'fi_Miller, Mcclure and Allen', 'fi_Nicholson Group', 'fi_OTHER',
       'fi_Richards-Walters', 'fi_Richardson Ltd',
       'fi_Romero, Woods and Johnson', 'fi_Sanchez, Hays and Wilkerson',
       'fi_Sanchez-Robinson', 'fi_Suarez Inc', 'fi_Swanson, Newton and Miller',
       'fi_Taylor, Hunt and Rodriguez', 'fi_Thornton-Davis',
       'fi_Turner, Baldwin and Rhodes', 'lp_A23', 'lp_B12', 'lp_C86'],
      dtype='object')

In [13]:
i_rates_freq = {}
for i,j in zip(df2['interest_rate'],train_y):
    if j==0:
        if i not in i_rates_freq.keys():
               i_rates_freq[i] = 1
        else:
               i_rates_freq[i] = i_rates_freq[i] + 1

keys = list(sorted(i_rates_freq))
i_rates_f = [i_rates_freq[i] for i in keys]

In [14]:
# keys

In [15]:
def divide_interests(num):
    counter = 1
    sum = 0
    di = {}
    for i in range(len(keys)):
        counter = counter + 1
        if counter == num:
#             print(keys[i])
            counter = 0
            if keys[i] not in di.keys():
                di[keys[i]] = sum
            sum = 0
        sum = sum + i_rates_f[i]

    labels = list(di.keys())
    values = list(di.values())
    
    return labels,values

In [16]:
from plotly.subplots import make_subplots
specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=2, cols=2, specs=specs)
night_colors = ['rgb(56, 75, 126)', 'rgb(18, 36, 37)', 'rgb(34, 53, 101)',
                'rgb(36, 55, 57)', 'rgb(6, 4, 4)']
# Define pie charts

ll = [(1,1),(1,2),(2,1),(2,2)]
vals = [46,92,115,230]
for i in range(len(vals)):
    labels,values = divide_interests(vals[i])
    fig.add_trace(go.Pie(labels=labels, values=values, marker_colors=night_colors), ll[i][0], ll[i][1])


fig = go.Figure(fig)
fig.show()

I have divided the analysis into different iteration step values. 
* Plot 1: Into 20 blocks
* Plot 2: Into 10 blocks
* Plot 3: Into 7 blocks
* Plot 4: Into 3 blocks

What we see is that majority of the loans having ROI below 3.8% get paid on time

In [17]:
count = 0
count2 = 0
passed_positive = 0
passed_zero = 0
i=0
for item in df2['co-borrower_credit_score']:
    if item == 0:
        count = count + 1
        if train_y[i] == 1:
            passed_zero = passed_zero + 1
    else:
        count2 = count2 + 1
        if train_y[i] == 1:
            passed_positive = passed_positive + 1
    i = i+1

In [18]:
count,count2

(47263, 68795)

In [19]:
passed_zero,passed_positive

(427, 209)

In [20]:
train_y.value_counts()

0    115422
1       636
Name: m13, dtype: int64

What we see is an imbalanced data. What we could try here is using SMOTE.

In [21]:
unique_ids = df2['loan_id']
df2.drop(columns = "loan_id",inplace = True)

In [22]:
df2.columns

Index(['interest_rate', 'unpaid_principal_bal', 'loan_term', 'loan_to_value',
       'number_of_borrowers', 'debt_to_income_ratio', 'borrower_credit_score',
       'insurance_percent', 'co-borrower_credit_score', 'insurance_type', 'm1',
       'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12',
       'response_time', 'source_X', 'source_Y', 'source_Z',
       'fi_Anderson-Taylor', 'fi_Browning-Hart', 'fi_Chapman-Mcmahon',
       'fi_Cole, Brooks and Vincent', 'fi_Edwards-Hoffman',
       'fi_Martinez, Duffy and Bird', 'fi_Miller, Mcclure and Allen',
       'fi_Nicholson Group', 'fi_OTHER', 'fi_Richards-Walters',
       'fi_Richardson Ltd', 'fi_Romero, Woods and Johnson',
       'fi_Sanchez, Hays and Wilkerson', 'fi_Sanchez-Robinson',
       'fi_Suarez Inc', 'fi_Swanson, Newton and Miller',
       'fi_Taylor, Hunt and Rodriguez', 'fi_Thornton-Davis',
       'fi_Turner, Baldwin and Rhodes', 'lp_A23', 'lp_B12', 'lp_C86'],
      dtype='object')

In [23]:
retained = []
from scipy.stats import pearsonr
for i in df2.columns:
    corr, _ = pearsonr(df2[i],train_y)
    if abs(corr)>0.01:
        retained.append(i)
        print(i+'\t\t\t\t '+str(corr))

interest_rate				 0.054297748337329886
unpaid_principal_bal				 -0.017119713693303878
loan_term				 0.02350150259625623
loan_to_value				 0.016356698057617143
number_of_borrowers				 -0.03997893193327681
debt_to_income_ratio				 0.0380332430271289
borrower_credit_score				 -0.09421547991626723
co-borrower_credit_score				 -0.04265141994916702
m1				 0.0920536306133328
m2				 0.14692067142997856
m3				 0.15739400237425377
m4				 0.21895630047719536
m5				 0.2595289494162428
m6				 0.28771179017698006
m7				 0.3090718669636777
m8				 0.33936180205435057
m9				 0.3691377263202238
m10				 0.3678395993744353
m11				 0.4102133930470283
m12				 0.4664416294225055
source_X				 -0.010544394670021748
source_Y				 0.010279241930136466
lp_A23				 -0.02553164234786174
lp_B12				 0.015562882468223672
lp_C86				 0.01398352210011473


In [24]:
retained

['interest_rate',
 'unpaid_principal_bal',
 'loan_term',
 'loan_to_value',
 'number_of_borrowers',
 'debt_to_income_ratio',
 'borrower_credit_score',
 'co-borrower_credit_score',
 'm1',
 'm2',
 'm3',
 'm4',
 'm5',
 'm6',
 'm7',
 'm8',
 'm9',
 'm10',
 'm11',
 'm12',
 'source_X',
 'source_Y',
 'lp_A23',
 'lp_B12',
 'lp_C86']

In [25]:
df3= df2[['interest_rate', 'unpaid_principal_bal', 'loan_term', 'loan_to_value','number_of_borrowers', 'debt_to_income_ratio','borrower_credit_score', 'co-borrower_credit_score',
 'm1',
 'm2',
 'm3',
 'm4',
 'm5',
 'm6',
 'm7',
 'm8',
 'm9',
 'm10',
 'm11',
 'm12',
 'source_X',
 'source_Y',
 'lp_A23',
 'lp_B12',
 'lp_C86']]

In [26]:
df3.columns

Index(['interest_rate', 'unpaid_principal_bal', 'loan_term', 'loan_to_value',
       'number_of_borrowers', 'debt_to_income_ratio', 'borrower_credit_score',
       'co-borrower_credit_score', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7',
       'm8', 'm9', 'm10', 'm11', 'm12', 'source_X', 'source_Y', 'lp_A23',
       'lp_B12', 'lp_C86'],
      dtype='object')

In [27]:
#Train 

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [28]:
import imblearn
from imblearn.over_sampling import SMOTE

smt = SMOTE()
X_train, y_train = smt.fit_sample(df3, train_y)

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1

In [29]:
X_train = pd.DataFrame(data = X_train, columns = df3.columns)

In [30]:
y_train = list(y_train)
np.bincount(y_train)

array([115422, 115422])

In [31]:
model = XGBClassifier(learning_rate = 0.05, max_depth = 4)
train_x, test_x, train_Y, test_y = train_test_split(X_train,y_train,train_size=0.8)
model.fit(train_x,train_Y)
xgb_predict = model.predict(test_x)
    
print( "Train Accuracy :: ", accuracy_score(train_Y, model.predict(train_x)))
print( "Test Accuracy  :: ", accuracy_score(test_y, xgb_predict))

Train Accuracy ::  0.9838039799648031
Test Accuracy  ::  0.9843401416534905


In [32]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [33]:
test_X = pd.read_csv("../input/india-ml-hiring-av/test.csv")

In [34]:
months = {"Feb":2,"Mar":3,"Apr":4,"May":5}
diff2 = []
for i in range(len(test_X['origination_date'])):
    temp = months[test_X['first_payment_date'][i].split('-')[0]] - int(test_X['origination_date'][i].split('/')[1])
    diff2.append(temp)
test_X.drop(columns = ['origination_date','first_payment_date'],inplace = True)
test_X['response_time'] = diff2

In [35]:
test_X = one_hot(test_X,['source','financial_institution','loan_purpose'],['source','fi','lp'])

In [36]:
test_ids = test_X['loan_id']
test_X = test_X[['interest_rate',
 'unpaid_principal_bal',
 'loan_term',
 'loan_to_value',
 'number_of_borrowers',
 'debt_to_income_ratio',
 'borrower_credit_score',
 'co-borrower_credit_score',
 'm1',
 'm2',
 'm3',
 'm4',
 'm5',
 'm6',
 'm7',
 'm8',
 'm9',
 'm10',
 'm11',
 'm12',
 'source_X',
 'source_Y',
 'lp_A23',
 'lp_B12',
 'lp_C86']]

In [37]:
df3.columns

Index(['interest_rate', 'unpaid_principal_bal', 'loan_term', 'loan_to_value',
       'number_of_borrowers', 'debt_to_income_ratio', 'borrower_credit_score',
       'co-borrower_credit_score', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7',
       'm8', 'm9', 'm10', 'm11', 'm12', 'source_X', 'source_Y', 'lp_A23',
       'lp_B12', 'lp_C86'],
      dtype='object')

In [38]:
test_X.columns

Index(['interest_rate', 'unpaid_principal_bal', 'loan_term', 'loan_to_value',
       'number_of_borrowers', 'debt_to_income_ratio', 'borrower_credit_score',
       'co-borrower_credit_score', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7',
       'm8', 'm9', 'm10', 'm11', 'm12', 'source_X', 'source_Y', 'lp_A23',
       'lp_B12', 'lp_C86'],
      dtype='object')

In [39]:
import joblib

In [40]:
# model = XGBClassifier()
# model.fit(df2, train_y)
# joblib.dump(model, "xgb1.joblib.dat")
# loaded_model = joblib.load("xgb1.joblib.dat")
# y_pred = model.predict(test_X)
# submission = pd.DataFrame(test_ids)
# submission['m13'] = y_pred
# submission.to_csv("mysubmission2.csv",index = False)
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier()

# for i in range(len([0,1,2,3])):
#     train_x, test_x, train_Y, test_y = train_test_split(df2,train_y,train_size=0.7)
#     model.fit(train_x,train_Y)
#     rfc_predict = model.predict(test_x)
    
#     print( "Train Accuracy :: ", accuracy_score(train_Y, model.predict(train_x)))
#     print( "Test Accuracy  :: ", accuracy_score(test_y, rfc_predict))

xgb_predict_test = model.predict(test_X)

In [41]:
submission = pd.DataFrame(test_ids)
submission['m13'] = xgb_predict_test
submission.to_csv("mysubmission7.csv",index = False)

In [42]:
submission

,loan_id,m13
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
5,6,0
6,7,0
7,8,0
8,9,0
9,10,0
